<a href="https://colab.research.google.com/github/cesphamm/procesamiento_lenguaje_natural/blob/main/Desafio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Procesamiento de lenguaje natural
## Desafío 3: Modelo de lenguaje con tokenización por caracteres

**Alumna:** Carla Espínola Hamm

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import urllib.request
import bs4 as bs

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from tensorflow.keras.utils import pad_sequences

# Configuración de estilo para gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.19.0
Keras version: 3.10.0


In [2]:
# Configuración de GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\nGPU detectada: {len(gpus)} dispositivo(s)")
    for gpu in gpus:
        print(f"   • {gpu.name}")

    # Habilitar crecimiento dinámico de memoria
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

    # Habilitar mixed precision para acelerar entrenamiento
    #tf.keras.mixed_precision.set_global_policy('mixed_float16')
    #print("\n Mixed Precision (float16) ACTIVADO")
    #print("Optimizaciones de GPU activadas")
else:
    print("\nNo se detectó GPU.")


GPU detectada: 1 dispositivo(s)
   • /physical_device:GPU:0


## 1. Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje

In [3]:
url = 'https://www.textos.info/miguel-de-cervantes-saavedra/el-ingenioso-hidalgo-don-quijote-de-la-mancha/ebook'
#url = 'https://www.textos.info/homero/odisea/ebook'
raw_html = urllib.request.urlopen(url)
raw_html = raw_html.read()

# Parsear artículo. 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

# Concatenar el texto de todos los párrafos
corpus = ''
for para in article_paragraphs:
    corpus += para.text + ' '

# Pasar todo el texto a minúscula
corpus = corpus.lower()

print(f"Longitud total del corpus: {len(corpus):,} caracteres")
print(f"\nPrimeros 500 caracteres del corpus:")
print("-" * 50)
print(corpus[:500])

Longitud total del corpus: 2,077,831 caracteres

Primeros 500 caracteres del corpus:
--------------------------------------------------
 yo, juan gallo de andrada, escribano de cámara del rey nuestro señor, de
los que residen en su consejo, certifico y doy fe que, habiendo visto por
los señores dél un libro intitulado el ingenioso hidalgo de la mancha,
compuesto por miguel de cervantes saavedra, tasaron cada pliego del dicho
libro a tres maravedís y medio; el cual tiene ochenta y tres pliegos, que
al dicho precio monta el dicho libro docientos y noventa maravedís y medio,
en que se ha de vender en papel; y dieron licencia para q


## 2. Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.

In [4]:
# Definir tamaño de contexto
MAX_CONTEXT_SIZE = 100

In [5]:
# Crear vocabulario de caracteres únicos
chars_vocab = sorted(set(corpus))
vocab_size = len(chars_vocab)

print(f"Tamaño del vocabulario: {vocab_size} caracteres únicos")
print(f"\nCaracteres en el vocabulario:")
print(chars_vocab)

Tamaño del vocabulario: 65 caracteres únicos

Caracteres en el vocabulario:
['\t', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '»', '¿', 'à', 'á', 'é', 'í', 'ï', 'ñ', 'ó', 'ù', 'ú', 'ü', '—']


In [6]:
# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {ch: idx for idx, ch in enumerate(chars_vocab)}
idx2char = {idx: ch for ch, idx in char2idx.items()}

print("🔗 Ejemplos de mapeo char2idx:")
for ch in ['a', 'e', 'i', 'o', 'u', ' ', '.']:
    if ch in char2idx:
        print(f"  '{ch}' -> {char2idx[ch]}")

🔗 Ejemplos de mapeo char2idx:
  'a' -> 25
  'e' -> 29
  'i' -> 33
  'o' -> 38
  'u' -> 44
  ' ' -> 2
  '.' -> 10


In [7]:
# Tokenizar el corpus completo
tokenized_corpus = np.array([char2idx[ch] for ch in corpus], dtype=np.int32)

print(f"Corpus tokenizado - shape: {tokenized_corpus.shape}")
print(f"\nPrimeros 50 tokens:")
print(tokenized_corpus[:50])

Corpus tokenizado - shape: (2077831,)

Primeros 50 tokens:
[ 2 48 38  8  2 34 44 25 37  2 31 25 35 35 38  2 28 29  2 25 37 28 41 25
 28 25  8  2 29 42 27 41 33 26 25 37 38  2 28 29  2 27 55 36 25 41 25  2
 28 29]


### Estructuración del Dataset

In [8]:
# separaremos el dataset entre entrenamiento y validación.
# `p_val` será la proporción del corpus que se reservará para validación
# `num_val` es la cantidad de secuencias de tamaño `MAX_CONTEXT_SIZE` que se usará en validación
p_val = 0.1
#num_val = int(np.ceil(len(tokenized_corpus)*p_val/MAX_CONTEXT_SIZE))
split_idx = int(len(tokenized_corpus) * (1 - p_val))

# separamos la porción de texto utilizada en entrenamiento de la de validación.
train_corpus = tokenized_corpus[:split_idx]
val_corpus = tokenized_corpus[split_idx:]

print(f"📊 División del corpus:")
print(f"  • Entrenamiento: {len(train_corpus):,} caracteres ({len(train_corpus)/len(tokenized_corpus)*100:.1f}%)")
print(f"  • Validación: {len(val_corpus):,} caracteres ({len(val_corpus)/len(tokenized_corpus)*100:.1f}%)")

📊 División del corpus:
  • Entrenamiento: 1,870,047 caracteres (90.0%)
  • Validación: 207,784 caracteres (10.0%)


In [9]:
def create_sequences(corpus_data, seq_length):
    """Crea secuencias de entrada y target para entrenamiento."""
    n_sequences = len(corpus_data) - seq_length
    X = np.zeros((n_sequences, seq_length), dtype=np.int32)
    y = np.zeros((n_sequences, seq_length), dtype=np.int32)
    for i in range(n_sequences):
        X[i] = corpus_data[i:i + seq_length]
        y[i] = corpus_data[i + 1:i + seq_length + 1]
    return X, y

X_train, y_train = create_sequences(train_corpus, MAX_CONTEXT_SIZE)
X_val, y_val = create_sequences(val_corpus, MAX_CONTEXT_SIZE)

print(f"Secuencias de entrenamiento: X={X_train.shape}, y={y_train.shape}")
print(f"Secuencias de validación: X={X_val.shape}, y={y_val.shape}")

Secuencias de entrenamiento: X=(1869947, 100), y=(1869947, 100)
Secuencias de validación: X=(207684, 100), y=(207684, 100)


In [10]:
# Crear tf.data.Dataset para entrenamiento
BATCH_SIZE = 1024 if gpus else 128
BUFFER_SIZE = 10000

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

# Crear secuencias de validación tokenizadas para PplCallback
num_val_sequences = len(val_corpus) // MAX_CONTEXT_SIZE
tokenized_sentences_val = [
    list(val_corpus[i * MAX_CONTEXT_SIZE:(i + 1) * MAX_CONTEXT_SIZE])
    for i in range(num_val_sequences)
]

print(f"Datasets creados:")
print(f"  • Batches de entrenamiento: {len(train_dataset)}")
print(f"  • Secuencias de validación para PPL: {len(tokenized_sentences_val)}")
print(f"  • Tamaño de batch: {BATCH_SIZE}")

Datasets creados:
  • Batches de entrenamiento: 1826
  • Secuencias de validación para PPL: 2077
  • Tamaño de batch: 1024


## 3. Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.

Probé embeddings como representación de los tokens, pero al no requerir aprendizaje, el tiempo de entrenamiento con one-hot encodding fue mucho más rápido y dado que  el vocabulario es pequeño (65 caracteres), se puede generar una representación directa sin pérdida de información. Para vocabularios grandes, los embeddings serían preferibles para reducir dimensionalidad.

Creo n capas de RNN buscando aumentar la capacidad de la red.

Uso dropuot para prevenir el overfitting que es bsantante posible en un corpus tan pequeño y normalización para estabilizar el entrenamiento y acelerar la convergencia.

### Función para arquitecturas

In [11]:
from keras.layers import Input, TimeDistributed, CategoryEncoding

def build_char_language_model(vocab_size, hidden_size=256, num_layers=2,
                               rnn_type='lstm', dropout=0.5, embedding_dim=128,
                               embed_dropout=0.2, train_type='one-hot'):
    """Construye un modelo de lenguaje a nivel de caracteres."""
    rnn_classes = {
        'rnn': layers.SimpleRNN,
        'lstm': layers.LSTM,
        'gru': layers.GRU
    }

    if rnn_type.lower() not in rnn_classes:
        raise ValueError(f"rnn_type debe ser 'rnn', 'lstm' o 'gru'")

    RNNLayer = rnn_classes[rnn_type.lower()]

    inputs = layers.Input(shape=(None,), dtype=tf.int32)

    if train_type.lower() == 'one-hot':
      # Reshape para CategoryEncoding: (batch, seq) -> (batch, seq, 1)
      x = layers.Reshape((-1, 1))(inputs)
      x = TimeDistributed(
          CategoryEncoding(num_tokens=vocab_size, output_mode="one_hot")
      )(x)
    else: ## Embeddings
      x = layers.Embedding(vocab_size, embedding_dim)(inputs)
      x = layers.Dropout(embed_dropout)(x)

    for i in range(num_layers):
      x = RNNLayer(
          hidden_size,
          return_sequences=True,
          dropout=dropout if i < num_layers - 1 else 0,
          recurrent_dropout=dropout if i < num_layers - 1 else 0
      )(x)

    x = layers.LayerNormalization()(x)
    x = layers.Dropout(dropout)(x)

    if train_type.lower() == 'one-hot':
      outputs = layers.Dense(vocab_size, activation='softmax', dtype='float32')(x)
    else: ## Embeddings
      outputs = layers.Dense(vocab_size, dtype='float32')(x)

    return Model(inputs=inputs, outputs=outputs)


In [ ]:
# Comparar arquitecturas
print("Comparación de arquitecturas en parámetros entrenables:")
print("=" * 50)
for rnn_type in ['rnn', 'lstm', 'gru']:
    model_temp = build_char_language_model(vocab_size, rnn_type=rnn_type)
    print(f"  {rnn_type.upper():>5}: {model_temp.count_params():>10,} parámetros")
    del model_temp
print("=" * 50)

### Entrenamiento de modelos

#### Utils

Uso la clase PplCallback provista por la cátedra con early stopping basado en perplexity, padding de secuencias y paciencia de 3.

In [13]:
class PplCallback(Callback):
    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, max_context_size, history_ppl, patience=3):
      # El callback lo inicializamos con secuencias de validación sobre las cuales
      # mediremos la perplejidad
      self.val_data = val_data
      self.max_context_size = max_context_size

      self.target = []
      self.padded = []

      count = 0
      self.info = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.patience = patience
      self.history_ppl = history_ppl

      # nos movemos en todas las secuencias de los datos de validación
      for seq in self.val_data:
          len_seq = len(seq)
          # armamos todas las subsecuencias
          subseq = [seq[:i] for i in range(1, len_seq)]
          self.target.extend([seq[i] for i in range(1, len_seq)])

          if len(subseq) != 0:
              self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))
              self.info.append((count, count + len_seq - 1))
              count += len_seq - 1

      if self.padded:
          self.padded = np.vstack(self.padded)


    def on_epoch_end(self, epoch, logs=None):
        if len(self.padded) == 0:
            print("\nNo hay datos de validación para calcular perplejidad")
            return
        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        # Calcular predicciones
        predictions = self.model.predict(self.padded, verbose=0)

        # Calcular perplejidad para cada secuencia para cada secuencia de validación
        for start, end in self.info:

          # en `probs` iremos guardando las probabilidades de los términos target
          probs = [predictions[idx_seq, -1, idx_vocab]
                     for idx_seq, idx_vocab in zip(range(start, end), self.target[start:end])]

          # calculamos la perplejidad por medio de logaritmos:  exp(-mean(log(probs)))
          if len(probs) > 0:
            scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores) if scores else np.inf
        self.history_ppl.append(current_score)
        train_ppl = np.exp(logs.get('loss', 0)) if logs else np.inf
        print(f"\n Epoch {epoch+1} | Train PPL: {train_ppl:7.2f} | Val PPL: {current_score:7.2f}", end='')

        # Early stopping basado en perplejidad
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save("best_model.keras")
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          print(f" (paciencia: {self.patience_counter}/{self.patience})")

          if self.patience_counter == self.patience:
            print("Early stopping por perplejidad...")
            self.model.stop_training = True

Construyo cada modelo según al arquitectura deseada, con RMSprop para optimización como es recomendado.
Definí la loss SparseCategoricalCrossentropy con  from_logits=False, dado que al usar one-hot ya se aplica softmax en la última capa de la red.

In [14]:
def train_model_keras(rnn_type, vocab_size, train_dataset, val_sequences,
                      max_context_size, hidden_size=256, num_layers=2,
                      embedding_dim=128, dropout=0.5, embed_dropout=0.2,
                      learning_rate=0.001, weight_decay=1e-5,
                      label_smoothing=0.1, num_epochs=30, patience=3,
                      train_type='one-hot'):
    """Entrena un modelo de lenguaje con early stopping."""
    print(f"\nIniciando entrenamiento - {rnn_type.upper()}")
    print("=" * 70)

    model = build_char_language_model(
        vocab_size, hidden_size, num_layers, rnn_type,
        dropout, embedding_dim, embed_dropout, train_type
    )

    optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate, weight_decay=weight_decay)

    if train_type.lower() == 'one-hot':
      loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    else: ## Embeddings
      loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True #, label_smoothing=label_smoothing
                                                         )

    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

    history_ppl = []
    callbacks = [
        PplCallback(
            val_data=val_sequences,
            max_context_size=max_context_size,
            history_ppl=history_ppl,
            patience=patience
        )
    ]

    history = model.fit(train_dataset,
                        epochs=num_epochs,
                        callbacks=callbacks,
                        verbose=1)

    history.history['val_perplexity'] = history_ppl
    best_ppl = min(history_ppl) if history_ppl else np.inf

    if history_ppl:
        model = keras.models.load_model("best_model.keras")

    print("=" * 70)
    print(f"Mejor perplejidad de validación: {best_ppl:.2f}")

    return model, history

#### Trains

A continuación, creo 3 arquitecturas vistas en clase: SimpleRNN (Celda de Elman), LSTM y GRU.

1. SimpleRNN:
   *   Será el baseline para evaluar arquitecturas más complejas.
   *   Es la más rápida en entrenar, ya que tiene menos parámetros y es la más simple.
   * Tiene el problema de vanishing gradients en secuencias largas y dificultad para capturar dependencias a largo plazo.
   * Resultado: PPL =

2. LSTM:
   * Las celdas de memoria permiten preservar información relevante.
   * Tiene mejor control de la información.
   * Es menos susceptible a vanishing gradients gracias a las conexiones residuales implícitas.
   * Resultado: PPL =


3. GRU:
   * Tiene menos parámetros que LSTM con rendimiento comparable y es más simple.
   * Es menos proclive al overfitting por tener menos parámetros y generaliza más en datasets chicos.
   * Resultado: PPL =

In [16]:
# Hiperparámetros
HIDDEN_SIZE = 256
NUM_LAYERS = 2
EMBEDDING_DIM = 128
DROPOUT = 0.5
EMBED_DROPOUT = 0.2
LEARNING_RATE = 0.001
NUM_EPOCHS = 30
PATIENCE = 3
WEIGHT_DECAY = 1e-5
LABEL_SMOOTHING = 0.1

models = {}
histories = {}

In [17]:
# Entrenar SimpleRNN
print("\n" + "="*70)
print("ENTRENANDO MODELO: SimpleRNN")
print("="*70)

models['rnn'], histories['rnn'] = train_model_keras(
    rnn_type='rnn',
    vocab_size=vocab_size,
    train_dataset=train_dataset,
    val_sequences=tokenized_sentences_val,
    max_context_size=MAX_CONTEXT_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    embedding_dim=EMBEDDING_DIM,
    dropout=DROPOUT,
    embed_dropout=EMBED_DROPOUT,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    label_smoothing=LABEL_SMOOTHING,
    num_epochs=NUM_EPOCHS,
    patience=PATIENCE,
    train_type='one-hot' #, train_type='embeddings'
)


ENTRENANDO MODELO: SimpleRNN

Iniciando entrenamiento - RNN
Epoch 1/30
1825/1826 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2005 - loss: 2.8626
 Epoch 1 | Train PPL:   14.49 | Val PPL:    9.70Saved new model!
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 125s 54ms/step - accuracy: 0.2006 - loss: 2.8624
Epoch 2/30
1825/1826 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2518 - loss: 2.5464
 Epoch 2 | Train PPL:   12.49 | Val PPL:    9.55Saved new model!
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 89s 49ms/step - accuracy: 0.2518 - loss: 2.5464
Epoch 3/30
1825/1826 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2649 - loss: 2.4907
 Epoch 3 | Train PPL:   12.07 | Val PPL:    9.15Saved new model!
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 88s 48ms/step - accuracy: 0.2649 - loss: 2.4907
Epoch 4/30
1825/1826 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2624 - loss: 2.5079
 Epoch 4 | Train PPL:   12.27 | Val PPL:   11.39 (paciencia: 1/3)
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 88s 48ms/step - accuracy: 0.2624 - loss: 2.5079
Epoch

In [ ]:
# Entrenar LSTM
print("\n" + "="*70)
print("ENTRENANDO MODELO: LSTM")
print("="*70)

models['lstm'], histories['lstm'] = train_model_keras(
    rnn_type='lstm',
    vocab_size=vocab_size,
    train_dataset=train_dataset,
    val_sequences=tokenized_sentences_val,
    max_context_size=MAX_CONTEXT_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    embedding_dim=EMBEDDING_DIM,
    dropout=DROPOUT,
    embed_dropout=EMBED_DROPOUT,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    label_smoothing=LABEL_SMOOTHING,
    num_epochs=NUM_EPOCHS,
    patience=PATIENCE,
    train_type='one-hot' #, train_type='embeddings'
)


ENTRENANDO MODELO: LSTM

Iniciando entrenamiento - LSTM
Epoch 1/30
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.2554 - loss: 2.5712
 Epoch 1 | Train PPL:   10.47 | Val PPL:    7.43Saved new model!
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 653s 354ms/step - accuracy: 0.2554 - loss: 2.5710
Epoch 2/30
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.3566 - loss: 2.1015
 Epoch 2 | Train PPL:    7.85 | Val PPL:    6.40Saved new model!
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 640s 350ms/step - accuracy: 0.3566 - loss: 2.1014
Epoch 3/30
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 0.3934 - loss: 1.9704
 Epoch 3 | Train PPL:    6.97 | Val PPL:    5.84Saved new model!
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 643s 352ms/step - accuracy: 0.3934 - loss: 1.9704
Epoch 4/30
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.4197 - loss: 1.8796
 Epoch 4 | Train PPL:    6.40 | Val PPL:    5.43Saved new model!
1826/1826 ━━━━━━━━━━━━━━━━━━━━ 644s 353ms/step - accuracy: 0.4197 - loss: 1.8795

In [ ]:
# Entrenar GRU
print("\n" + "="*70)
print("ENTRENANDO MODELO: GRU")
print("="*70)

models['gru'], histories['gru'] = train_model_keras(
    rnn_type='gru',
    vocab_size=vocab_size,
    train_dataset=train_dataset,
    val_sequences=tokenized_sentences_val,
    max_context_size=MAX_CONTEXT_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    embedding_dim=EMBEDDING_DIM,
    dropout=DROPOUT,
    embed_dropout=EMBED_DROPOUT,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    label_smoothing=LABEL_SMOOTHING,
    num_epochs=NUM_EPOCHS,
    patience=PATIENCE,
    train_type='one-hot' #, train_type='embeddings'
)

In [ ]:
colors = {'rnn': '#e74c3c', 'lstm': '#3498db', 'gru': '#2ecc71'}
labels_map = {'rnn': 'SimpleRNN', 'lstm': 'LSTM', 'gru': 'GRU'}

if histories:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Subplot 0: Perplexity de validación (solo cuando exista)
    for model_type, history in histories.items():
        val_ppl = history.history.get('val_perplexity', None)
        if val_ppl is None:
            val_loss = history.history.get('val_loss', None)
            if val_loss is not None:
                val_ppl = [float(np.exp(l)) for l in val_loss]
        if val_ppl:
            epochs_val = range(1, len(val_ppl) + 1)
            axes[0].plot(epochs_val, val_ppl, color=colors.get(model_type, None),
                         label=labels_map.get(model_type, model_type), linewidth=2,
                         marker='o', markersize=4)
    axes[0].set_xlabel('Época')
    axes[0].set_ylabel('Perplejidad')
    axes[0].set_title('Perplejidad de Validación')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Subplot 1: curvas de loss (train vs val)
    for model_type, history in histories.items():
        loss = history.history.get('loss', [])
        if loss:
            epochs_train = range(1, len(loss) + 1)
            axes[1].plot(epochs_train, loss, color=colors.get(model_type, None),
                         label=f"{labels_map.get(model_type, model_type)} (train)", linestyle='-')

        val_loss = history.history.get('val_loss', None)
        val_ppl = history.history.get('val_perplexity', None)
        if val_ppl:
            epochs_val = range(1, len(val_ppl) + 1)
            axes[1].plot(epochs_val, val_ppl, color=colors.get(model_type, None),
                         label=f"{labels_map.get(model_type, model_type)} (val PPL)", linestyle='--')
        elif val_loss:
            epochs_val = range(1, len(val_loss) + 1)
            axes[1].plot(epochs_val, val_loss, color=colors.get(model_type, None),
                         label=f"{labels_map.get(model_type, model_type)} (val loss)", linestyle='--')

    axes[1].set_xlabel('Época')
    axes[1].set_ylabel('Loss / PPL')
    axes[1].set_title('Curvas de Aprendizaje')

    handles, labels = axes[1].get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    axes[1].legend(by_label.values(), by_label.keys())
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("\nResumen de Modelos:")
    print("=" * 55)
    for model_type, history in histories.items():
        val_ppl = history.history.get('val_perplexity', None)
        if val_ppl:
            best_ppl = min(val_ppl)
        else:
            val_loss = history.history.get('val_loss', None)
            if val_loss:
                best_ppl = float(np.exp(min(val_loss)))
            else:
                best_ppl = np.inf
        if np.isfinite(best_ppl):
            print(f"  {labels_map.get(model_type, model_type):<10}: PPL = {best_ppl:.2f}")
        else:
            print(f"  {labels_map.get(model_type, model_type):<10}: PPL = N/A (sin datos de validación)")
    print("=" * 55)


## 4. Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.

In [ ]:
# Seleccionar mejor modelo
best_model_type = min(histories, key=lambda x: min(histories[x].history.get('val_perplexity', [np.inf])))

model = models[best_model_type]
best_ppl = min(histories[best_model_type].history.get('val_perplexity', [np.inf]))

print(f"Usando modelo: {best_model_type.upper()} (PPL: {best_ppl:.2f})")

### Utils

In [ ]:
def greedy_search(model, seed_text, max_length, num_chars):
    """Genera texto usando búsqueda greedy."""
    generated_text = seed_text.lower()
    for _ in range(num_chars):
        tokens = [char2idx.get(ch, 0) for ch in generated_text[-max_length:]]
        if len(tokens) < max_length:
            tokens = [0] * (max_length - len(tokens)) + tokens
        #x = np.array([tokens], dtype=np.int32)
        x = pad_sequences([tokens], maxlen=max_length, padding='pre')
        logits = model.predict(x, verbose=0)
        next_char_idx = np.argmax(logits[0, -1, :])
        generated_text += idx2char[next_char_idx]
    return generated_text

In [ ]:
def sample_with_temperature(model, seed_text, max_length, num_chars, temperature=1.0):
    """Genera texto usando muestreo con temperatura."""
    generated_text = seed_text.lower()
    for _ in range(num_chars):
        tokens = [char2idx.get(ch, 0) for ch in generated_text[-max_length:]]
        if len(tokens) < max_length:
            tokens = [0] * (max_length - len(tokens)) + tokens
        #x = np.array([tokens], dtype=np.int32)
        x = pad_sequences([tokens], maxlen=max_length, padding='pre')

        logits = model.predict(x, verbose=0)
        logits_scaled = logits[0, -1, :] / temperature
        probs = tf.nn.softmax(logits_scaled).numpy()
        next_char_idx = np.random.choice(len(probs), p=probs)
        generated_text += idx2char[next_char_idx]
    return generated_text

In [ ]:
def beam_search_deterministic(model, seed_text, max_length, num_chars, beam_width=5):
    """Genera texto usando beam search determinístico."""
    seed_text = seed_text.lower()
    beams = [(seed_text, 0.0)]
    for _ in range(num_chars):
        all_candidates = []
        for text, score in beams:
            tokens = [char2idx.get(ch, 0) for ch in text[-max_length:]]
            if len(tokens) < max_length:
                tokens = [0] * (max_length - len(tokens)) + tokens
            #x = np.array([tokens], dtype=np.int32)
            x = pad_sequences([tokens], maxlen=max_length, padding='pre')

            logits = model.predict(x, verbose=0)
            log_probs = tf.nn.log_softmax(logits[0, -1, :]).numpy()
            top_indices = np.argsort(log_probs)[-beam_width:]
            for idx in top_indices:
                all_candidates.append((text + idx2char[idx], score + log_probs[idx]))
        all_candidates.sort(key=lambda x: x[1], reverse=True)
        beams = all_candidates[:beam_width]
    final_sequences = [(text, score / len(text)) for text, score in beams]
    final_sequences.sort(key=lambda x: x[1], reverse=True)
    return final_sequences[0][0], final_sequences

In [ ]:
def beam_search_stochastic(model, seed_text, max_length, num_chars, beam_width=5, temperature=1.0):
    """Genera texto usando beam search estocástico."""
    seed_text = seed_text.lower()
    beams = [(seed_text, 0.0)]
    for _ in range(num_chars):
        all_candidates = []
        for text, score in beams:
            tokens = [char2idx.get(ch, 0) for ch in text[-max_length:]]
            if len(tokens) < max_length:
                tokens = [0] * (max_length - len(tokens)) + tokens
            #x = np.array([tokens], dtype=np.int32)
            x = pad_sequences([tokens], maxlen=max_length, padding='pre')

            logits = model.predict(x, verbose=0)
            logits_scaled = logits[0, -1, :] / temperature
            probs = tf.nn.softmax(logits_scaled).numpy()
            log_probs = np.log(probs + 1e-10)
            sampled_indices = np.random.choice(len(probs), size=min(beam_width, len(probs)),
                                               replace=False, p=probs)
            for idx in sampled_indices:
                all_candidates.append((text + idx2char[idx], score + log_probs[idx]))
        all_candidates.sort(key=lambda x: x[1], reverse=True)
        beams = all_candidates[:beam_width]
    final_sequences = [(text, score / len(text)) for text, score in beams]
    final_sequences.sort(key=lambda x: x[1], reverse=True)
    return final_sequences[0][0], final_sequences

### Análisis

1. GREEDY
    * Es determinístico. Siempre produce el mismo resultado para la misma semilla.
    * Es muy rápido porque solo elige el más probable en cada paso.
    * Tiene tendencia a responder loops.

2. BEAM SEARCH
    * Selecciona los mejores n beams y avanza al siguiente nivel repitiendo el proceso.
    * Selecciona la secuencia con mejor score normalizado.
    * Sigue siendo determinístico, aunque mejor que GREEDY.
3. BEAM STOCHASTIC
    * Muestrea k opciones según una probabilidad dada.
    * Genera textos diferentes en cada ejecución.
    * La temperatura permite ajustar creatividad.

In [ ]:
# Ejemplos de generación
seed = "ulises dijo"
print("="*70)
print(f"COMPARACIÓN DE MÉTODOS - Semilla: '{seed}'")
print("="*70)

print("\nGREEDY:")
print(greedy_search(model, seed, MAX_CONTEXT_SIZE, 100))

#print("\nSAMPLING (T=0.7):")
#print(sample_with_temperature(model, seed, MAX_CONTEXT_SIZE, 100, 0.7))

print("\nBEAM SEARCH (width=5):")
result, _ = beam_search_deterministic(model, seed, MAX_CONTEXT_SIZE, 100, 5)
print(result)

print("\nBEAM STOCHASTIC (width=5, T=0.7):")
result, _ = beam_search_stochastic(model, seed, MAX_CONTEXT_SIZE, 100, 5, 0.7)

print(result)

La temperatura es el hiperparámetro más importante para controlar el estilo de generación: valores bajos para más determinismo, valores altos para exploración creativa.

T	| Comportamiento esperado
--|--
0.2	| Texto muy repetitivo pero gramaticalmente correcto
0.5	| Buen equilibrio, texto fluido
0.8	| Más creativo, ocasionales sorpresas
1.0	| Distribución original del modelo
1.5	| Alta creatividad, posibles errores

In [ ]:
# Efecto de la temperatura
seed = "el héroe regresó"
print("="*70)
print(f"EFECTO DE LA TEMPERATURA - Semilla: '{seed}'")
print("="*70)

for temp in [0.2, 0.5, 0.8, 1.0, 1.5]:
    print(f"\nT = {temp}:")
    print(beam_search_stochastic(model, seed, MAX_CONTEXT_SIZE, 80, 5, temp))


In [ ]:
# Guardar modelo
model.save('best_char_lm_keras.keras')